<h1> Task 1 </h1>
<h3> 2. Load the three JSON files and encode them as pandas dataframes. </h3>

In [1]:
import pandas as pd

In [2]:
test_url = '/Users/kor/Desktop/Unibo/II year/Fall/NLP/Project/Assignment 1/data/test.json'
training_url = '/Users/kor/Desktop/Unibo/II year/Fall/NLP/Project/Assignment 1/data/training.json'
validation_url = '/Users/kor/Desktop/Unibo/II year/Fall/NLP/Project/Assignment 1/data/validation.json'

In [3]:
training_set = pd.read_json(training_url, orient='index')
validation_set = pd.read_json(validation_url, orient='index')
test_set = pd.read_json(test_url, orient='index')

<h3> 3. Generate hard labels for Task 1 using majority voting and store them in a new dataframe column called `hard_label_task1`. <br>Items without a clear majority will be removed from the dataset. </h3>

In [4]:
def majority(l):
    y_count = l.count('YES')
    n_count = l.count('NO')

    if y_count == n_count:
        return pd.NaT
    
    if y_count > 3:
        return 'YES'

    return 'NO'

In [5]:
training_set['hard_label_task1'] = training_set['labels_task1'].apply(majority)
training_set.dropna(axis=0, inplace=True)

validation_set['hard_label_task1'] = validation_set['labels_task1'].apply(majority)
validation_set.dropna(axis=0, inplace=True)

test_set['hard_label_task1'] = test_set['labels_task1'].apply(majority)
test_set.dropna(axis=0, inplace=True)

<h3> 4. Filter the DataFrame to keep only rows where the `lang` column is `'en'`. </h3>

In [6]:
lang = 'en'

In [7]:
training_set = training_set[training_set['lang'] == lang]
validation_set = validation_set[validation_set['lang'] == lang]
test_set = test_set[test_set['lang'] == lang]

<h3> 5. Remove unwanted columns: Keep only `id_EXIST`, `lang`, `tweet`, and `hard_label_task1`. </h3>

In [8]:
training_set = training_set.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
validation_set = validation_set.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
test_set = test_set.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]

<h3> 6. Encode the `hard_label_task1` column: Use 1 to represent "YES" and 0 to represent "NO".</h3>

In [9]:
training_set['hard_label_task1'] = training_set['hard_label_task1'].map({'YES':1, 'NO':0})
validation_set['hard_label_task1'] = validation_set['hard_label_task1'].map({'YES':1, 'NO':0})
test_set['hard_label_task1'] = test_set['hard_label_task1'].map({'YES':1, 'NO':0})

In [10]:
training_set

,id_EXIST,lang,tweet,hard_label_task1
200002,200002,en,Writing a uni essay in my local pub with a cof...,1
200003,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,1
200006,200006,en,According to a customer I have plenty of time ...,1
200007,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",1
200008,200008,en,New to the shelves this week - looking forward...,0
...,...,...,...,...
203256,203256,en,idk why y’all bitches think having half your a...,1
203257,203257,en,This has been a part of an experiment with @Wo...,1
203258,203258,en,"""Take me already"" ""Not yet. You gotta be ready...",1
203259,203259,en,@clintneedcoffee why do you look like a whore?...,1


<h1> Task 2 </h1>

- **Remove emojis** from the tweets.
- **Remove hashtags** (e.g., `#example`).
- **Remove mentions** such as `@user`.
- **Remove URLs** from the tweets.
- **Remove special characters and symbols**.
- **Remove specific quote characters** (e.g., curly quotes).
- **Perform lemmatization** to reduce words to their base form.

In [ ]:
import re
import emoji
from nltk.corpus import stopwords
import nltk

In [ ]:
emojis_list = map(lambda x: ''.join(x.split()), emoji.EMOJI_DATA.keys())

EMOJI_RE = re.compile('|'.join(re.escape(p) for p in emojis_list))
HASHTAGS_RE = re.compile('#\w+')
MENTIONS_RE = re.compile('@\w+')
URL_RE = re.compile('(https|http)?:\/\/\S+')
#Qui ho aggiunto il punto sostituito dallo spazio perchè mi sembra che la maggior parte 
#dei tweets ne traggono beneficio, poi TODO va provato raga
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;‘’“”\"\.]')
SPECIAL_CHARACTERS_RE = re.compile('[&amp;]')
GOOD_SYMBOLS_RE = re.compile('[^\w+ +]')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

In [ ]:
def remove_emojis(text: str) -> str:
    return EMOJI_RE.sub(' ',text)

def remove_hashtags(text: str) -> str:
    return HASHTAGS_RE.sub(' ', text)

def remove_mentions(text: str) -> str:
    return MENTIONS_RE.sub(' ', text)

def remove_url(text: str) -> str:
    return URL_RE.sub(' ',text)

def remove_special_characters(text: str) -> str:
    return SPECIAL_CHARACTERS_RE.sub('', text)

def replace_special_characters(text: str) -> str:
    return REPLACE_BY_SPACE_RE.sub(' ', text)

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the good symbols list (check regular expression)
    """
    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    """
    return text.strip()

We can observe that some hashtags in the form "#somethinghttps://" also removes the initial part of the link.

@rufinelix's account

In [ ]:
from typing import List, Callable, Dict
from functools import reduce

PREPROCESSING_PIPELINE = [
                          remove_emojis,
                          remove_hashtags,
                          remove_mentions,
                          remove_url,
                          remove_special_characters,
                          replace_special_characters,
                          filter_out_uncommon_symbols,
                          strip_text
                          ]
#Lui elimina anche le stopwords, poi TODO va provato raga

def text_prepare(text: str,
                 filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

In [ ]:
print('Pre-processing text...')

print()
print(f'[Debug] Before:\n{training_set.tweet.values[2]}')
print()

# Replace each sentence with its pre-processed version
training_set['tweet'] = training_set['tweet'].apply(lambda txt: text_prepare(txt))
validation_set['tweet'] = validation_set['tweet'].apply(lambda txt: text_prepare(txt))
test_set['tweet'] = test_set['tweet'].apply(lambda txt: text_prepare(txt))

print(f'[Debug] After:\n{training_set.tweet.values[2]}')
print()

print("Pre-processing completed!")

Pre-processing text...

[Debug] Before:
According to a customer I have plenty of time to go spent the Stirling coins he wants to pay me with, in Derry. "Just like any other woman, I'm sure of it." #EveryDaySexism in retail.

[Debug] After:
According to a customer I have plenty of time to go spent the Stirling coins he wants to pay me with  in Derry  Just like any other woman  Im sure of it    in retail

Pre-processing completed!


In [ ]:
#Focus Focus Focus qui le contrazioni tipo I'm le accorpa, ma sotto la lemmatization con wordNet le 
#scoppia. Se usassimo le stopwords le eliminerebbe direttamente e il problema non si porrebbe, quindi indovinate
#TODO va provato raga
training_set.iloc[:10]['tweet']

200002    Writing a uni essay in my local pub with a cof...
200003    it is 2021 not 1921 I dont appreciate that on ...
200006    According to a customer I have plenty of time ...
200007    So only blokes drink beer Sorry  but if you ar...
200008    New to the shelves this week  looking forward ...
200010       I guess that s fairly normal for a Neanderthal
200011    means women usually end up in lower paid suppo...
200013    Hi Orla Interesting piece On No 2 As a policy ...
200015    Dear GodColette is capable of identifying sexi...
200016    The women will all be at home cooking for the ...
Name: tweet, dtype: object

In [65]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import WhitespaceTokenizer

lemmatizer = WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()

def get_wordnet_key(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return 'n'

def lem_text(text: str):
    tokens = tokenizer.tokenize(text)
    tagged = pos_tag(tokens)
    words = [lemmatizer.lemmatize(word, get_wordnet_key(tag)) for word, tag in tagged]
    return " ".join(words)

In [ ]:
lem_train_texts = [lem_text(text) for text in training_set['tweet']]
lem_validation_texts = [lem_text(text) for text in validation_set['tweet']]
lem_test_texts = [lem_text(text) for text in test_set['tweet']]

In [72]:
lem_train_texts[:10]

['Writing a uni essay in my local pub with a coffee Random old man keep ask me drunk question when Im try to concentrate amp end with good luck but youll just end up get married and not use it anyway be alive and well',
 'it be 2021 not 1921 I dont appreciate that on two ride by myself your team member look behind me and ask the man behind how many in my party Not impressed',
 'According to a customer I have plenty of time to go spent the Stirling coin he want to pay me with in Derry Just like any other woman Im sure of it in retail',
 'So only blokes drink beer Sorry but if you arent a bloke you drink wine apparently alive and well in',
 'New to the shelf this week look forward to read these book',
 'I guess that s fairly normal for a Neanderthal',
 'mean woman usually end up in low paid support work when that start to change traditionalist notice unfairness that previously hasnt bother them at all',
 'Hi Orla Interesting piece On No 2 As a policy response do you believe earlyinlife p

<h1> Task 3 </h1>

Embed words using **GloVe embeddings**. <br>
You are **free** to pick any embedding dimension.